In [2]:
import glob
from string import punctuation

original_sentences = []
simplified_sentences = []

def remove_tokenization_artifacts(s, src):
    stokens = s.split()
    snew = s
    for i, token in enumerate(stokens):
        if i > 0 and i < len(stokens) - 1 and token in punctuation:
            substrboth = stokens[i - 1] + token + stokens[i + 1]
            substrleft = stokens[i - 1] + token
            substright = token + stokens[i + 1]
            if substrboth in src:
                snew = snew.replace(stokens[i - 1] + " " + token + " " + stokens[i + 1], substrboth)
            elif substrleft in src:
                snew = snew.replace(stokens[i - 1] + " " + token, substrleft)
            elif substright in src:
                snew = snew.replace(token + " " + stokens[i + 1], substright)

    snew = snew.replace("''", '"')
    snew = snew.replace(" .", ".")
    snew_rest = "" if len(snew) == 1 else snew[1:]
    if len(snew) > 0:
        snew = snew[0].capitalize() + snew_rest
    snew = snew.replace("-lrb-", "(").replace("-rrb-", ")")
    snew = snew.replace("-LRB-", "(").replace("-RRB-", ")")
    return snew


# read ./systems/asset.test.orig
with open("systems/asset.test.orig") as f:
    sentences = [sentence.strip() for sentence in f.readlines()]
    original_sentences = sentences

outputs_dict = {}

# read each file in systems directory
for file in glob.glob("systems/*"):
    if file == "systems/three_new_systems_input.json":
        continue
    # read the file by line
    with open(file) as f:
        # get the system name from file
        # system = file.split("/")[-1]
        sentences = f.readlines()
        # remove newline character from each line
        sentences = [sentence.strip() for sentence in sentences]
        for i, simplified in enumerate(sentences):
            original_sentence = original_sentences[i]
            if original_sentence not in outputs_dict:
                outputs_dict[original_sentence] = {}
            outputs_dict[original_sentence][file] = remove_tokenization_artifacts(simplified, original_sentence)

In [3]:
import json
# save outputs_dict to json file
with open("18_systems_dict.json", "w") as f:
    json.dump(outputs_dict, f)
